In [1]:
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm
import random

def stopping_distance(speed, next_speed):
    if next_speed > speed:
        return 0
    n = 0
    for i in range(next_speed, speed + 1, 1):
        n += i
    return int(n)

MAX_SPEED = 10
MAX_FORESIGHT = int((MAX_SPEED * (MAX_SPEED - 1)) / 2)

NOTHING = 0
ACCELERATE = 1
DECCELERATE = 2

def get_most_restrictive_car(street, index, speed):
    min_stopping_dist = len(street)
    foresight = stopping_distance(speed, 0)
    
    for j in range(1, foresight + 1):
        if street[(index + j) % len(street)] >= 0:
            # We've found a car
            speed2 = street[(index + j) % len(street)]
            stopping_dist = stopping_distance(speed2, 0) + (j - 1)
            if stopping_dist < min_stopping_dist:
                min_stopping_dist = stopping_dist

    if min_stopping_dist > foresight:
        return ACCELERATE
    elif min_stopping_dist < foresight:
        return DECCELERATE
    else:
        return NOTHING
    
                

def get_next_car(street, index):
    distance_to_next_car = MAX_FORESIGHT
    speed_of_next_car = MAX_SPEED
    for j in range(1, MAX_FORESIGHT):
        if street[(index + j) % len(street)] >= 0:
            distance_to_next_car = j - 1
            speed_of_next_car = street[(index + j) % len(street)]
            break
            
    return distance_to_next_car, speed_of_next_car
    

class Lane:
    def __init__(self, cell_count=120, max_speed=8):
        assert(max_speed <= MAX_SPEED)
        self.cell_count = cell_count
        self.max_speed = max_speed
        self.street = -np.ones(self.cell_count, dtype=np.byte)
    
    def set_density(self, density, speed=2):
        assert(speed <= self.max_speed)
        accrual = 0.5
        for i in range(self.cell_count):
            accrual += density
            if accrual > 1:
                self.street[i] = speed
                accrual -= 1
                
    def print_out(self):
        textual_representation = ""
        for i in range(len(self.street)):
            if self.street[i] < 0:
                textual_representation += "."
            else:
                textual_representation += str(self.street[i])

        print(textual_representation)

    def tick(self, input_arr=None, output_arr=None):
        new_street = -np.ones(self.cell_count, dtype=np.byte)
        for i in range(self.cell_count):
            if self.street[i] >= 0:
                # Found car
                speed = self.street[i]

                # Find next car ahead
                distance_to_next_car, speed_of_next_car = get_next_car(self.street, i)
                action =  get_most_restrictive_car(self.street, i, speed+1)

                new_speed = speed
                if action == ACCELERATE and speed < self.max_speed:
                    # Press on accelerator
                    new_speed += 1
                elif action == DECCELERATE:
                    # Press on the brakes
                    new_speed -= 1


                max_forward = min(speed, distance_to_next_car + speed_of_next_car)
                if max_forward < speed:
                    # Rear end
                    new_speed = 0

                new_speed = max(0, min(new_speed, self.max_speed))
                new_street[(i + max_forward) % len(self.street)] = new_speed

        self.street = new_street
    


In [2]:
lane = Lane(50, 8)
lane.set_density(0.1)
lane.print_out()
lane.tick()
lane.print_out()

.....2.........2.........2.........2.........2....
.......3.........3.........3.........3.........3..


In [12]:
def print_lane(lane):
    textual_representation = ""
    for i in range(len(lane)):
        if lane[i] < 0:
            textual_representation += "."
        else:
            textual_representation += str(lane[i])

    print(textual_representation)

def arr_to_road(array):
    textual_representation = ""
    for i in range(len(array)):
        if array[i] < 0:
            textual_representation += "."
        else:
            textual_representation += str(array[i])

    return textual_representation
    
def stopping_distance(speed, next_speed):
    if next_speed > speed:
        return 0
    n = 0
    for i in range(next_speed, speed + 1, 1):
        n += i
    return int(n)

MAX_SPEED = 10
MAX_FORESIGHT = int((MAX_SPEED * (MAX_SPEED - 1)) / 2)

NOTHING = 0
ACCELERATE = 1
DECCELERATE = 2

def get_most_restrictive_car(street, index, speed):
    min_stopping_dist = len(street)
    foresight = stopping_distance(speed, 0)
    
    for j in range(1, foresight + 1):
        if index + j >= len(street):
            break
        if street[(index + j)] >= 0:
            # We've found a car
            speed2 = street[(index + j)]
            stopping_dist = stopping_distance(speed2, 0) + (j - 1)
            if stopping_dist < min_stopping_dist:
                min_stopping_dist = stopping_dist

    if min_stopping_dist > foresight:
        return ACCELERATE
    elif min_stopping_dist < foresight:
        return DECCELERATE
    else:
        return NOTHING
    
                

def get_next_car(street, index):
    distance_to_next_car = MAX_FORESIGHT
    speed_of_next_car = MAX_SPEED
    for j in range(1, MAX_FORESIGHT):
        if index + j >= len(street):
            break
        if street[(index + j)] >= 0:
            distance_to_next_car = j - 1
            speed_of_next_car = street[(index + j)]
            break
            
    return distance_to_next_car, speed_of_next_car

def update_lane(input_segment, lane, output_segment, max_speed=MAX_SPEED):
    full_lane = lane
    input_len = 0
    output_len = 0
    if input_segment is not None:
        input_len = len(input_segment)
        full_lane = np.concatenate([input_segment, full_lane], axis=0)
    if output_segment is not None:
        output_len = len(output_segment)
        full_lane = np.concatenate([full_lane, output_segment], axis=0)
    
    total_length = len(full_lane)

    # Keep track of what cars leave off the end
    leaving_cars = [] # [(speed, distance)]
    
    new_street = -np.ones(total_length, dtype=np.byte)
    for i in range(total_length):
        if full_lane[i] >= 0:
            # Found car
            speed = full_lane[i]

            # Find next car ahead
            distance_to_next_car, speed_of_next_car = get_next_car(full_lane, i)
            action =  get_most_restrictive_car(full_lane, i, speed+1)

            new_speed = speed
            if action == ACCELERATE and speed < max_speed:
                # Press on accelerator
                new_speed += 1
            elif action == DECCELERATE:
                # Press on the brakes
                new_speed -= 1


            max_forward = min(speed, distance_to_next_car + speed_of_next_car)
            if max_forward < speed:
                # Rear end
                new_speed = 0

            new_speed = max(0, min(new_speed, max_speed))
            if (i + max_forward) < total_length:
                new_street[(i + max_forward)] = new_speed
            else:
                leaving_cars.append((new_speed, i + max_forward - total_length))

    if input_len != 0 and output_len != 0:
        return (new_street[:input_len], new_street[input_len:-output_len], new_street[-output_len:], leaving_cars)
    elif output_len != 0:
        return (None, new_street[:-output_len], new_street[-output_len:], leaving_cars)
    elif input_len != 0:
        return (new_street[:input_len], new_street[input_len:], None, leaving_cars)
    else:
        return (None, new_street, None, leaving_cars)

In [13]:
class Lane:
    def __init__(self, cell_count, max_speed, start_x=0, start_y=0, end_x=0, end_y=0):
        self.cell_count = cell_count
        self.max_speed = max_speed
        self.array = -np.ones(cell_count, dtype=np.byte)
        
        self.start_x = start_x
        self.start_y = start_y
        self.end_x = end_x
        self.end_y = end_y
        
    def set_density(self, density, speed=2):
        assert(speed <= self.max_speed)
        accrual = 0.5
        for i in range(self.cell_count):
            accrual += density
            if accrual > 1:
                self.array[i] = speed
                accrual -= 1
          
    def get_average_speed(self, segments):
        speeds = []
        percent_step = 1.0 / segments
        prev_index = 0
        for i in range(segments):
            start_index = round(percent_step * i * self.cell_count)
            end_index = round(percent_step * (i + 1) * self.cell_count)
            
            sub_arr = self.array[start_index:end_index]
        
            if sum(sub_arr != -1) == 0:
                speeds.append(0)
            else:
                speeds.append(sum(sub_arr[sub_arr != -1]) / sum(sub_arr != -1))
        return speeds
    
    def get_average_density(self, segments):
        densities = []
        percent_step = 1.0 / segments
        prev_index = 0
        for i in range(segments):
            start_index = round(percent_step * i * self.cell_count)
            end_index = round(percent_step * (i + 1) * self.cell_count)
            
            sub_arr = self.array[start_index:end_index]
        
            if sum(sub_arr != -1) == 0:
                densities.append(0)
            else:
                densities.append(sum(sub_arr != -1) / len(sub_arr))
        return densities
                
    def tick(self, input_segment=None, output_segment=None):
        new_input, self.array, new_output, leaving_cars = update_lane(input_segment, self.array, output_segment, max_speed=self.max_speed)
        return new_input, new_output, leaving_cars

    def get_string(self):
        return arr_to_road(self.array)
        
    def print_out(self):
        print(self.get_string())

In [14]:
lane = Lane(80, 8)
lane.set_density(0.05)
lane.print_out()
lane.tick()
lane.print_out()
lane.tick()
lane.print_out()
lane.tick()
lane.print_out()
lane.tick()
lane.print_out()
lane.tick()
lane.print_out()
lane.tick()
lane.print_out()
lane.tick()
lane.print_out()
lane.tick()
lane.print_out()
lane.tick()
lane.print_out()
lane.get_average_density(20)

.........2...................2...................2...................2..........
...........3...................3...................3...................3........
..............4...................4...................4...................4.....
..................5...................5...................5...................5.
.......................6...................6...................6................
.............................7...................7...................7..........
....................................8...................8...................8...
............................................8...................8...............
....................................................8...................8.......
............................................................8...................


[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0.25, 0, 0, 0, 0]

In [6]:
# Intersections must keep track many aspects about a car.
class Car:
    def __init__(self, in_speed, in_direction):
        self.in_speed = in_speed
        self.age = 0
        
    def set_desired_direction(self, direction):
        self.direction = direction
        
    def age_car(self):
        self.age += 1

In [7]:
import random
import numpy.random as np_random
import math

NORTH_INDEX = 0
EAST_INDEX = 1
SOUTH_INDEX = 2
WEST_INDEX = 3

class NoInlet():
    def __init__(self):
        pass
    
    def tick(self, intersection, inlet_index):
        return []

class LaneInlet():
    def __init__(self, lane):
        self.lane = lane
        
    # Returns the cars that enter the intersection
    def tick(self, intersection, inlet_index):
        output_segment = intersection.get_waiting_array(inlet_index)
        stopped_cars = sum(output_segment == 0)
            
        _, new_output, leaving_cars = self.lane.tick(output_segment=output_segment)
        
        new_cars = []
        for i in range(stopped_cars, len(output_segment)):
            if new_output[i] != -1:
                new_cars.append(Car(new_output[i], inlet_index))
        
        # Return the ones that went in first
        return list(reversed(new_cars))

class StochasticInlet():
    def __init__(self, rate, speed=4):
        self.gamma = 1.0 / rate
        self.timer = np_random.exponential(self.gamma, size=1)[0]
        self.speed = speed
        
    # Returns the cars that enter the intersection
    def tick(self, intersection, inlet_index):
        self.timer -= 1
        if self.timer < 0:
            self.timer = np_random.exponential(self.gamma, size=1)[0]
            return [Car(self.speed, inlet_index)]
        else:
            return []
    
class ConstantInlet():
    def __init__(self, rate, speed=4):
        self.cycles_between = 1.0 / rate
        self.timer = self.cycles_between * random.random()
        self.speed = speed
        
    # Returns the cars that enter the intersection
    def tick(self, intersection, inlet_index):
        self.timer -= 1
        if self.timer < 0:
            self.timer = self.cycles_between
            return [Car(self.speed, inlet_index)]
        else:
            return []
        
# This outlet allows every car to leave
class AllOutlet():
    def __init__(self):
        pass
    
    # Return the array of cars that haven't left
    def tick(self, cars_leaving):
        return []
    
class LaneOutlet():
    def __init__(self, lane):
        self.lane = lane
    
    # Return the array of cars that haven't left
    def tick(self, cars_leaving):
        # Car has just executed it's trajectory, add it to the lane
        room = 0
        for i in range(MAX_SPEED):
            if self.lane.array[i] == -1:
                room += 1
            else:
                break
        
        cars_leaving_count = len(cars_leaving)
        leaving = min(room, cars_leaving_count)
        if leaving > 0:
            spacing = math.floor(room / cars_leaving_count)

            idx = 0
            for i in range(0, room, spacing):
                car_leave_speed = 1
                if cars_leaving[idx].age == 0:
                    car_leave_speed = cars_leaving[idx].in_speed
                    
                self.lane.array[i] = car_leave_speed # Leave at constant rate
                idx += 1
            
        if (cars_leaving_count - leaving) == 0:
            return []
        else:
            return cars_leaving[:-(cars_leaving_count - leaving)]

In [10]:
# Intersection modeling:
# Update all roads
# Update intersection logic

NORTH_SOUTH_GREEN = 0
EAST_WEST_GREEN = 1
        
class Intersection():
    def __init__(self):
        self.inlets = [NoInlet(), NoInlet(), NoInlet(), NoInlet()]
        self.outlets = [AllOutlet(), AllOutlet(), AllOutlet(), AllOutlet()]
        self.waiting = [[], [], [], []]
        self.leaving = [[], [], [], []]
        self.light_state = EAST_WEST_GREEN
        self.light_time_waiting = 30
        
    def set_inlet(self, index, inlet):
        self.inlets[index] = inlet
        
    def set_outlet(self, index, outlet):
        self.outlets[index] = outlet
        
    def print_out(self):
        print(f"NORTH: {self.inlets[0]} {self.outlets[0]}")
        print(f"SOUTH: {self.inlets[1]} {self.outlets[1]}")
        print(f"EAST : {self.inlets[2]} {self.outlets[2]}")
        print(f"WEST : {self.inlets[3]} {self.outlets[3]}")
        
    # Returns the array of cars waiting
    # 2 cars waiting => [-1, -1, -1, ... -1, 0, 0]
    def get_waiting_array(self, index, length=MAX_SPEED):
        waiting_array = -np.ones(length, dtype=np.byte)
        cars_waiting = len(self.waiting[index])
        for i in range(cars_waiting):
            waiting_array[i] = 0
            
        return waiting_array
        
    def tick_in(self):
        # Get all new cars that come in from intersection
        for i in range(4):
            if self.inlets[i] != None:
                new_cars = self.inlets[i].tick(self, i)
                for j in range(len(new_cars)):
                    # Figure out which way they car will go
                    LEFT_CHANCE = 0.1
                    STRAIGHT_CHANCE = 0.65
                    RIGHT_CHANCE = 0.25
                    
                    decision = random.random()
                    if decision < LEFT_CHANCE:
                        new_cars[j].set_desired_direction((i - 1) % 4) # Turn Left
                    elif decision < LEFT_CHANCE + STRAIGHT_CHANCE:
                        new_cars[j].set_desired_direction((i - 2) % 4) # Go Straight
                    else:
                        new_cars[j].set_desired_direction((i + 1) % 4) # Turn Right
                    
                    self.waiting[i].append(new_cars[j])

    def tick_out(self):
        # Update where each car is leaving
        for in_dir in range(4):
            # TODO: This only goes straight
            
            # Check if any cars are waiting to go at the intersection
            if len(self.waiting[in_dir]) > 0:
                out_dir = self.waiting[in_dir][0].direction
                
                if self.light_state == NORTH_SOUTH_GREEN and (in_dir == NORTH_INDEX or in_dir == SOUTH_INDEX):
                    if len(self.waiting[in_dir]) != 0 and len(self.leaving[out_dir]) == 0: # And can leave
                        car_going = self.waiting[in_dir].pop(0)
                        self.leaving[out_dir].append(car_going)
                        
                elif self.light_state == EAST_WEST_GREEN and (in_dir == EAST_INDEX or in_dir == WEST_INDEX):
                    if len(self.waiting[in_dir]) != 0 and len(self.leaving[out_dir]) == 0: # And can leave
                        car_going = self.waiting[in_dir].pop(0)
                        self.leaving[out_dir].append(car_going)
                        
            # Update ages
            for j in range(len(self.waiting[in_dir])):
                self.waiting[in_dir][j].age_car()
        
        
        # Update each outlet
        for i in range(4):
            if len(self.leaving[i]) != 0:
                self.leaving[i] = self.outlets[i].tick(self.leaving[i])
        
        # Update traffic light state
        self.light_time_waiting -= 1
        if self.light_time_waiting <= 0:
            self.light_state, self.light_time_waiting = self.get_next_light_state()
            
    def get_next_light_state(self):
        if self.light_state == NORTH_SOUTH_GREEN:
            return EAST_WEST_GREEN, 30
        if self.light_state == EAST_WEST_GREEN:
            return NORTH_SOUTH_GREEN, 30
            
        
intersection1 = Intersection()
lane1 = Lane(50, 4)
lane1_ = Lane(50, 4)
intersection2 = Intersection()
lane2 = Lane(50, 4)
lane2_ = Lane(50, 4)
intersection3 = Intersection()

intersection1.set_inlet(WEST_INDEX, StochasticInlet(0.5))
intersection1.set_inlet(EAST_INDEX, LaneInlet(lane1_))
intersection1.set_outlet(EAST_INDEX, LaneOutlet(lane1))
intersection1.set_outlet(WEST_INDEX, AllOutlet())

intersection2.set_inlet(WEST_INDEX, LaneInlet(lane1))
intersection2.set_inlet(EAST_INDEX, LaneInlet(lane2_))
intersection2.set_outlet(EAST_INDEX, LaneOutlet(lane2))
intersection2.set_outlet(WEST_INDEX, LaneOutlet(lane1_))

intersection3.set_inlet(WEST_INDEX, LaneInlet(lane2))
intersection3.set_inlet(EAST_INDEX, StochasticInlet(0.5))
intersection3.set_outlet(EAST_INDEX, AllOutlet())
intersection3.set_outlet(WEST_INDEX, LaneOutlet(lane2_))

for i in range(1000):
    intersection1.tick_in()
    intersection2.tick_in()
    intersection3.tick_in()
    
    intersection1.tick_out()
    intersection2.tick_out()
    intersection3.tick_out()
    
    print(lane1.get_string() + " " + arr_to_road(intersection2.get_waiting_array(WEST_INDEX)) + " " + lane2.get_string())
#     print(lane1_.get_string()[::-1] + " " + arr_to_road(intersection2.get_waiting_array(EAST_INDEX))[::-1] + " " + lane2_.get_string()[::-1])
#     print("")


.................................................. .......... ..................................................
4................................................. .......... ..................................................
....4............................................. .......... ..................................................
........4......................................... .......... ..................................................
4...........4..................................... .......... ..................................................
....4...........4................................. .......... ..................................................
........4...........4............................. .......... ..................................................
............4...........4......................... .......... ..................................................
4...............4...........4..................... .......... ..................................

1..........................................0000000 .......... 1.............................................0000
12.........................................0000001 .......... .2............................................0001
11.3.......................................000000. .......... 2..3..........................................000.
.02...4....................................000001. .......... ..3...4.......................................001.
.1..3.....4................................00000.2 .......... .....4....4...................................00.2
1.2....4......4............................00001.. .......... 3........3....4...............................01..
.2..3......4......4........................0000.2. .......... ...4........4.....4...........................0.2.
...3...4.......4......4....................0001... .......... 3......4........4.....4.......................1...
......4....4.......4......4................000.2.. .......... ...4.......4........4.....4.......

................................................00 0......... ................................................00
................................................00 0......... ................................................00
................................................00 0......... ................................................00
................................................00 0......... ................................................00
................................................00 0......... ................................................00
................................................00 0......... ................................................00
1...............................................00 .......... ................................................00
.2..............................................01 .......... ................................................01
1..3............................................0. .......... 2.................................

In [9]:
def lerp(a: float, b: float, t: float) -> float:
    return (1 - t) * a + t * b

class Grid:
    def __init__(self, x_grid, y_grid, length, max_speed):
        self.x_grid = x_grid
        self.y_grid = y_grid
        self.max_speed = max_speed
        self.grid = np.zeros(shape=(x_grid, y_grid), dtype=object)
        self.lanes = []
        
        for x in range(x_grid):
            for y in range(y_grid):
                self.grid[x,y] = Intersection()
                self.grid[x,y].set_inlet(NORTH_INDEX, StochasticInlet(0.5))
                self.grid[x,y].set_inlet(SOUTH_INDEX, StochasticInlet(0.5))
                self.grid[x,y].set_inlet(EAST_INDEX, StochasticInlet(0.5))
                self.grid[x,y].set_inlet(WEST_INDEX, StochasticInlet(0.5))

        for x in range(x_grid):
            for y in range(y_grid):
                ST_WIDTH = 0.05
                if x != 0:
                    lane = Lane(length, max_speed, start_x=x-ST_WIDTH, start_y=y+ST_WIDTH, end_x=x-1+ST_WIDTH, end_y=y+ST_WIDTH)
                    self.grid[x-1,y].set_inlet(WEST_INDEX, LaneInlet(lane))
                    self.grid[x,y].set_outlet(EAST_INDEX, LaneOutlet(lane)) # Right
                    
                    lane2 = Lane(length, max_speed, start_x=x-1+ST_WIDTH, start_y=y-ST_WIDTH, end_x=x-ST_WIDTH, end_y=y-ST_WIDTH)
                    self.grid[x,y].set_inlet(EAST_INDEX, LaneInlet(lane2))
                    self.grid[x-1,y].set_outlet(WEST_INDEX, LaneOutlet(lane2)) # Left
                    
                    self.lanes.append(lane)
                    self.lanes.append(lane2)
                    
                if y != 0:
                    lane = Lane(length, max_speed, start_x=x-ST_WIDTH, start_y=y-ST_WIDTH, end_x=x-ST_WIDTH, end_y=y-1+ST_WIDTH)
                    self.grid[x,y-1].set_inlet(NORTH_INDEX, LaneInlet(lane))
                    self.grid[x,y].set_outlet(SOUTH_INDEX, LaneOutlet(lane)) # Down
                    
                    lane2 = Lane(length, max_speed, start_x=x+ST_WIDTH, start_y=y-1+ST_WIDTH, end_x=x+ST_WIDTH, end_y=y-ST_WIDTH)
                    self.grid[x,y].set_inlet(SOUTH_INDEX, LaneInlet(lane2))
                    self.grid[x,y-1].set_outlet(NORTH_INDEX, LaneOutlet(lane2)) # Up
                    
                    self.lanes.append(lane)
                    self.lanes.append(lane2)
                    
    def tick(self):
        for x in range(self.x_grid):
            for y in range(self.y_grid):
                self.grid[x,y].tick_in()
                
        for x in range(self.x_grid):
            for y in range(self.y_grid):
                self.grid[x,y].tick_out()
                
                
    def draw(self, ax):
        for i in range(len(self.lanes)):
            lane = self.lanes[i]
            speeds = lane.get_average_density(40)
            for i in range(len(speeds)):
                perc_start = i / len(speeds)
                perc_end = (i + 1) / len(speeds)
            
                c = speeds[i]
                xs = [lerp(lane.start_x, lane.end_x, perc_start), lerp(lane.start_x, lane.end_x, perc_end)]
                ys = [lerp(lane.start_y, lane.end_y, perc_start), lerp(lane.start_y, lane.end_y, perc_end)]
                ax.plot(xs, ys, c=(c,0.0,0.0), linewidth=4.0)


In [ ]:
%matplotlib notebook

import numpy as np
import matplotlib.pyplot as plt
import time

city_grid = Grid(4, 4, 40, 5)

fig,ax = plt.subplots(1,1, figsize=(8, 8))
for f in range(600):
    ax.set_title(f"Frame {str(f).zfill(3)}")
    print(f"Frame {str(f).zfill(3)}")
    for i in range(1):
        city_grid.tick()
    city_grid.draw(ax)
    plt.savefig(f'frames/frame{str(f).zfill(3)}.png')


<IPython.core.display.Javascript object>

Frame 000
Frame 001
Frame 002
Frame 003
Frame 004
Frame 005
Frame 006
Frame 007
Frame 008
Frame 009
Frame 010
Frame 011
Frame 012
Frame 013
Frame 014
Frame 015
Frame 016
Frame 017
Frame 018
Frame 019
Frame 020
Frame 021
Frame 022
Frame 023
Frame 024
Frame 025
Frame 026
Frame 027
Frame 028
Frame 029
Frame 030
Frame 031
Frame 032
Frame 033
Frame 034
Frame 035
Frame 036
Frame 037
Frame 038
Frame 039
Frame 040
Frame 041
Frame 042
Frame 043
Frame 044
Frame 045
Frame 046
Frame 047
Frame 048
Frame 049
Frame 050
Frame 051
Frame 052
Frame 053
Frame 054
Frame 055
Frame 056
Frame 057
Frame 058
Frame 059
Frame 060
Frame 061
Frame 062
Frame 063
Frame 064
Frame 065
Frame 066
Frame 067
Frame 068
Frame 069
Frame 070
Frame 071
Frame 072
Frame 073
Frame 074
Frame 075
Frame 076
Frame 077
Frame 078
Frame 079
Frame 080
Frame 081
Frame 082
Frame 083
Frame 084
Frame 085
Frame 086
Frame 087
Frame 088
Frame 089
Frame 090
Frame 091
Frame 092
Frame 093
Frame 094
Frame 095
Frame 096
Frame 097
Frame 098
Frame 099


In [12]:
inlet = StochasticInlet(0.5)